In [ ]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

# 重み付き平均MAPE表の作成@CG

In [ ]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)
input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_cg["functionName"])))

In [ ]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

# 重み付き平均MAPE表の作成@MG

In [ ]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

mg_input_list_problem_size: list[int] = [32, 64, 128, 256]
mg_input_list_nit: list[int] = [4, 10, 20, 35]

mg_target_list_problem_size: list[int] = [512]
mg_target_list_nit: list[int] = [50]

# input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=input_list_process,
#     list_na=cg_input_list_na,
#     list_nonzer=cg_input_list_nonzer,
#     list_niter=cg_input_list_niter,
#     list_shift=cg_input_list_shift,
#     csvDir=csvDirPath,
# )
# input_rawDF_cg = input_rawDF_cg.rename(columns={"Name":"functionName"})

input_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=input_list_process,
    list_problem_size=mg_input_list_problem_size,
    list_nit=mg_input_list_nit,
    csvDir=csvDirPath,
)
input_rawDF_mg = input_rawDF_mg.rename(columns={"Name": "functionName"})


# target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=target_list_process,
#     list_na=cg_target_list_na,
#     list_nonzer=cg_target_list_nonzer,
#     list_niter=cg_target_list_niter,
#     list_shift=cg_target_list_shift,
#     csvDir=csvDirPath,
# )
# target_rawDF_cg = target_rawDF_cg.rename(columns={"Name":"functionName"})

target_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=target_list_process,
    list_problem_size=mg_target_list_problem_size,
    list_nit=mg_target_list_nit,
    csvDir=csvDirPath,
)
target_rawDF_mg = target_rawDF_mg.rename(columns={"Name": "functionName"})

mg_list_exp: list[str] = ["process", "problem_size", "nit"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_mg["functionName"])))

In [ ]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_mg[
        input_rawDF_mg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_mg[
        target_rawDF_mg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=mg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")
    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[mg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_mg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

In [ ]:
retNum_mg

In [ ]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

In [ ]:
date: str = "2022年8月26日"


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

targetFunctionNames: list[str] = [
    "VECSET",
    "MPI_Wait()",
    "MPI_Send()",
    "SPRNVC",
    "RANDLC",
    "MPI_Irecv()",
    "ICNVRT",
]

function_names: list[str] = list(set(input_rawDF_cg["Name"].tolist()))

input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})


list_modelName_dict: dict[str, str] = {
    "modelIp": "線形",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る",
    "modelProblemSizeDividedByProcess": "問題サイズをプロセス数で割る",
    "modelLinearSumOf2elementCombination": "全変数の組合せ積の総和",
    "modelLinearSumOfElementCombinations": "全変数の組合せ積の線形和",
    "modelLinearSumOf2elementCombinationWithSquared": "2要素の積の線形和（2乗含む）",
    "modelLinearSumOf2elementCombinationWithCubed": "2要素の積の線形和（3乗含む）",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses": "問題サイズの変数の総乗とプロセス数とをかける",
    "modelInfiniteProductOfProblemSizeDividedByProcesses": "問題サイズの変数の総乗をプロセス数で割る",
    "modelSquareRootOfProcess": "プロセス数の平方根",
    "modelSquareRootTimesOtherElems": "プロセス数の平方根と他説明変数の積",
    "modelObeyOneParameter": "一つのパラメータに依存する",
    "modelLin": "線形モデル"
    # "modelBasicTree",
}

In [ ]:
result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    # "modelSquareRootOfProcess",
    # "modelSquareRootTimesOtherElems",
    # "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_before = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_before

In [ ]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_before: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

In [ ]:
print(
    resultDF_before.loc[:, ["functionName", "最低値", "最適モデル"]]
    .sort_values("functionName")
    .rename(columns={"functionName": "関数名", "最低値": "重み付きMAPE"})
    .replace(list_modelName_dict)
    .style.to_latex(label=f"{date}_resultDF_before", caption=f"（修正前）", siunitx=True)
)

print(f"retNum_cg_before = {retNum_cg_before}")

In [ ]:
result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_after = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_after

In [ ]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

# 重み付き平均MAPEの算出@CG

In [ ]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_after: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

In [ ]:
print(
    resultDF_after.loc[:, ["functionName", "最低値", "最適モデル"]]
    .sort_values("functionName")
    .rename(columns={"functionName": "関数名", "最低値": "重み付きMAPE"})
    .replace(list_modelName_dict)
    .style.to_latex(label=f"{date}_resultDF_before", caption=f"（修正後）", siunitx=True)
)

print(f"retNum_cg_after = {retNum_cg_after}")

In [ ]:
date: str = "2022年8月26日"

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)
input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_cg["functionName"])))

In [ ]:
list_series: list[pd.Series] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    # "modelLinearSumOf2elementCombinationWithSquared",
    # "modelLinearSumOf2elementCombinationWithCubed",
    # "modelSquareRootOfProcess",
    # "modelSquareRootTimesOtherElems",
    # "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_before: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

print(f"retNum_cg_before={retNum_cg_before}")

In [ ]:
list_series: list[pd.Series] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_after: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

print(f"retNum_cg_after={retNum_cg_after}")

In [ ]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

In [ ]:
date: str = "2022年9月2日"

target_list_process: list[int] = [256]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_niter=cg_target_list_niter,
    list_nonzer=cg_target_list_nonzer,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg

In [ ]:
# cg_target_list_na: list[int] = [1500000]
# cg_target_list_nonzer: list[int] = [21]
# cg_target_list_niter: list[int] = [100]
# cg_target_list_shift: list[int] = [200]

get_execTime(
    benchmarkName="cg",
    condition={"process": 256, "na": 1500000, "nonzer": 21, "niter": 100, "shift": 200},
    csvDir=csvDirPath,
)

In [ ]:
test_convertPprofTime()

In [ ]:
target_rawDF_cg["Exclusive"]

_before_converted = list(target_rawDF_cg["Exclusive"])

_after_converted = map(convertPprofTime, _before_converted)

_after_converted

target_rawDF_cg["converted_Exclusive"] = list(_after_converted)

target_rawDF_cg

In [ ]:
sum(target_rawDF_cg["converted_Exclusive"])

In [ ]:
test_ret_relativeCost()

In [ ]:
import csv


benchmarkName: str = "cg"
conditions: dict[str, list[int]] = {
    "process": [2, 4],
    "na": [14000, 75000],
    "nonzer": [11, 21],
    "niter": [90, 100],
    "shift": [60, 80],
}
ret = gen_ExtraPinputData(
    benchmarkName=benchmarkName, conditions=conditions, csvDir=csvDirPath
)
print(ret)

In [ ]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

In [ ]:
# 元データによる結果を取得する
# ✅1. Extra-Pに読み込ますデータの取得
# 2. 取得できた数式から予測


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

str_ExtraPinputData: str = gen_ExtraPinputData(
    benchmarkName="cg",
    conditions={
        "process": input_list_process,
        "na": cg_input_list_na,
        "nonzer": cg_input_list_nonzer,
        "niter": cg_input_list_niter,
        "shift": cg_input_list_shift,
    },
)

filePath = "./extra-p_docker/share/input_cg.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

In [ ]:
%sx pwd
%sx which extrap
%ls
%sx extrap --text /workspace/extra-p_docker/share/input_cg.txt

In [ ]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

str_ExtraPinputData: str = gen_ExtraPinputData(
    benchmarkName="lulesh",
    conditions={
        "process": train_lulesh_processes,
        "iteration": train_lulesh_iterations,
        "size": train_lulesh_sizes,
    },
    version=2,
)

filePath = "./extra-p_docker/share/input_lulesh.txt"
with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

In [ ]:
%sx extrap --text /workspace/extra-p_docker/share/input_lulesh.txt

In [ ]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

# TODO

1. ✅CG, MGで時間をExtra-Pから取得できることを確認
2. ✅時間の単位を「秒」にする
    * 列「Inclusive total msec」に一部存在している”：”で区切られた値の単位は、msec ではなく分と秒を区切る”：”
3. ✅相対コストの算出関数の実装
4. ✅Extra-P のDocker環境の構築
4. Extra-P による予測との組み合わせを行う
    * なにをどう組み合わせるのかがわかっていないので、それは確かめる
    * 元データ, Extra-P単体で予測したデータ, <何か> で結果を作成

# 予測結果に必要なもの

* ✅元データ（予測環境の生データ）
* ✅Extra-P単体で予測したデータ
* Extra-Pで各関数の実行時間を予測し、それにコール回数を掛けた値で予測したデータ
    * ✅コール回数で実行時間を割る
    * ✅Extra-Pでモデルを作成 <- コール回数で実行時間を割る
    * ✅👆を利用して予測
    * ✅本プログラムでモデルを作成 <- コール回数で実行時間を割る
    * ✅👆を利用して予測
    * ✅「コール回数 * 実行時間」を計算して予測

上記のセルよりモデルは次式

$$ 2.5521930556315375 + 0.013737041505459383 * iteration + 3.41165137739477 * 10^7 * iteration * size^3 * \log_2 size $$

In [ ]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]

In [ ]:
trainDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=train_lulesh_processes,
    list_iteration=train_lulesh_iterations,
    list_size=train_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
trainDF_lulesh["InclusivePerCall"] = -1
trainDF_lulesh = trainDF_lulesh.reset_index()
for i, sr in trainDF_lulesh.iterrows():
    trainDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(
        sr["Inclusive"]
    ) / float(sr["#Call"])
trainDF_lulesh = trainDF_lulesh.sort_values("#Call")
# Inclusive列の整形
_before_converted: list[float] = list(trainDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
trainDF_lulesh["Inclusive"] = list(_after_converted)

trainDF_lulesh

In [ ]:
testDF_lulesh: pd.DataFrame = return_rawDF_lulesh(
    list_process=test_lulesh_processes,
    list_iteration=test_lulesh_iterations,
    list_size=test_lulesh_sizes,
    csvDir=csvDirPath,
)

# InclusivePerCall列の生成
testDF_lulesh["InclusivePerCall"] = -1
testDF_lulesh = testDF_lulesh.reset_index()
for i, sr in testDF_lulesh.iterrows():
    testDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(sr["Inclusive"]) / float(
        sr["#Call"]
    )
testDF_lulesh = testDF_lulesh.sort_values("#Call")

# Inclusive列の整形
_before_converted: list[float] = list(testDF_lulesh["Inclusive"])
_after_converted: list[float] = map(convertPprofTime, _before_converted)
testDF_lulesh["Inclusive"] = list(_after_converted)

testDF_lulesh

In [ ]:
# ✅関数ごとにfor文を回す
#   * 今回は試験的に１種類の関数で実施
# ✅ExtraP向けの入力データを作成（総実行時間）
# ✅ExtraP向けの入力データを作成（１回実行あたり）
# ✅ExtraPでモデル構築（総実行時間）
# ✅ExtraPでモデル構築（１回実行あたり）
# ✅ExtraPで時間を予測（総実行時間）
# ✅ExtraPで時間を予測（１回実行あたり）
# 関数コール回数予測のためのモデルを構築
# 関数コール回数を予測
# 予測された実行時間と予測されたコール回数をまとめる

functionNames: list[str] = sorted(list(set(trainDF_lulesh["Name"])))
for functionName in functionNames:
    pass

functionName = "Real_t_CalcElemVolume(const"
trainDF_perFunc: pd.DataFrame = trainDF_lulesh[trainDF_lulesh["Name"] == functionName]
testDF_perFunc: pd.DataFrame = testDF_lulesh[testDF_lulesh["Name"] == functionName]

expVar: list[str] = ["process", "iteration", "size"]
resVar: str

In [ ]:
# 総実行時間

In [ ]:
resVar = "Inclusive"
str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
    inputDF=trainDF_perFunc,
    expVar=expVar,
    resVar=resVar,
)

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

%sx extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep -v Loading | grep -v Measurement

$$ 総実行時間 =  0.008938888890764687 + 0.0003391531590741526 * iteration + 2.645514630961448 * 10^{-8} * iteration * size^3 * \log_2{size} $$

In [ ]:
# 上式より予測対象環境での総実行時間は
time_all = (
    0.008938888890764687
    + 0.0003391531590741526 * test_lulesh_iterations[0]
    + 2.645514630961448e-08
    * test_lulesh_iterations[0]
    * test_lulesh_sizes[0] ** (3)
    * np.log2(test_lulesh_sizes[0])
)
print(time_all)

In [ ]:
# 1回実行あたり

In [ ]:
resVar = "InclusivePerCall"
str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
    inputDF=trainDF_perFunc,
    expVar=expVar,
    resVar=resVar,
)

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

%sx extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt | grep -v Loading | grep -v Measurement

$$ 一回当たりの実行時間 = 1.5456768003451272 * 10^{-7} $$

In [ ]:
# 上式より、予測対象環境での1回あたりの実行時間は
time_perCall = 1.5456768003451272 * 10 ** (-7)
print(time_perCall)

In [ ]:
# 関数コール回数予測のためのモデルを構築

resVar = "#Call"

result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

trainDF_perFunc = trainDF_perFunc.rename({"Name": "functionName"}, axis="columns")
testDF_perFunc = testDF_perFunc.rename({"Name": "functionName"}, axis="columns")

models = Models(
    inputDF=trainDF_perFunc,
    expVarColNames=expVar,
    resVarColNames=[resVar],
    targetDF=None,
    modelNames=list_modelName,
)

models.setUpDataBeforeCalcLr()
models.calcLr()
models.calcMAPE()


dictCalcedMAPE = models.returnCalculatedMAPE()
for key in dictCalcedMAPE.keys():
    dictCalcedMAPE[key] = float(dictCalcedMAPE[key])
dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
dict_for_series["functionName"] = functionName

series: pd.Series = pd.Series(dict_for_series)
result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_after = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)


resultDF_after

In [ ]:
bestModelDict: dict = return_bestModelObject(
    inputDF=trainDF_perFunc,
    list_expVar=expVar,
    list_resVar=[resVar],
    list_modelName=list_modelName,
)

bestModel = bestModelDict["object"]

predicted = float(np.array(bestModel.predict(inputDF=testDF_perFunc[expVar])))
_call: float = float(testDF_perFunc.iloc[0][resVar])
_MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
_series: pd.Series = pd.Series(
    {
        "functionName": functionName,
        "call": _call,
        "MAPE": _MAPE,
        "predicted_call": predicted,
    }
)

call = _call

_series

In [ ]:
testDF_perFunc

In [ ]:
real_time = testDF_lulesh.loc[0]["Inclusive"]

print(
    f"real_time = {real_time}\ntime_all = {time_all}\ntime_perCall * call= {time_perCall * call}\n"
)

# Extra-P の出力の制御に関する実験

In [ ]:
from sympy import sympify, pprint, symbols, log
from sympy.parsing.sympy_parser import parse_expr

eq1: str = """0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log2(size)^(1)"""
eq2: str = """1.5456768003451272e-07"""

In [ ]:
dict_symbols = {}
for elem in expVar:
    dict_symbols[elem] = symbols(elem, real=True)
dict_symbols

In [ ]:
loaded_eq1 = sympify(eq1, locals=dict_symbols)

loaded_eq2 = sympify(eq2, locals=dict_symbols)

In [ ]:
print(type(loaded_eq1))
loaded_eq1

In [ ]:
ans = loaded_eq1.subs(
    [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
    ]
)
ans

In [ ]:
ans.evalf()

In [ ]:
print(type(ans))

In [ ]:
eq = sympify("log2(size) + iteration + size", locals=dict_symbols)
ans_eq = eq.subs(
    [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
    ]
)
print(type(ans_eq))
print(type(ans_eq.evalf()))
print(ans_eq.evalf())

In [ ]:
eq = sympify("ln(size)/ln(2) + iteration + size", locals=dict_symbols)
ans_eq = eq.subs(
    [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
    ]
)
print(type(ans_eq))
print(type(ans_eq.evalf()))
print(ans_eq.evalf())

In [ ]:
eq = sympify("ln(2) + iteration + size", locals=dict_symbols)
ans_eq = eq.subs(
    [
        (dict_symbols["size"], test_lulesh_sizes[0]),
        (dict_symbols["iteration"], test_lulesh_iterations[0]),
    ]
)
print(type(ans_eq))
print(type(ans_eq.evalf()))
print(ans_eq.evalf())

In [ ]:
type(ans_eq.evalf())

In [ ]:
loaded_eq2

In [ ]:
import re

inputStr: str = """0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log2(size)^(1)"""

pattern = re.compile(r"log(?P<base>[0-9])")

result = pattern.sub(r"1/ln(\g<base>)ln", inputStr)
print(result)

In [ ]:
import re

inputStr: str = """0.008938888890764687 + 0.0003391531590741526 * iteration^(1) + 2.645514630961448e-08 * iteration^(1) * size^(3) * log25(size)^(1)"""

pattern = re.compile(r"log(?P<base>\d{1,})")

result = pattern.sub(r"1/ln(\g<base>)ln", inputStr)
print(result)

In [ ]:
resVar = "Inclusive"
str_ExtraPinputData: str = gen_ExtraPinputDataFromDF(
    inputDF=trainDF_perFunc,
    expVar=expVar,
    resVar=resVar,
)

filePath: str = f"./extra-p_docker/share/input_lulesh_perFunc.txt"

with open(filePath, mode="w") as f:
    f.write(str_ExtraPinputData)

%sx extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model

# Extra-P をpythonから実行し、出力を受け取る実験

In [ ]:
import subprocess

try:
    res = subprocess.check_output("pwd")
except:
    print("Error.")
print(res)

try:
    res = subprocess.check_output("ls")
except:
    print("Error.")
print(res)


res_str: str = subprocess.run(
    "extrap --text ./extra-p_docker/share/input_lulesh_perFunc.txt  | grep Model",
    stdout=subprocess.PIPE,
    text=True,
    shell=True,
).stdout

In [ ]:
res_str = res_str.replace("Model: ", "")
print(res_str)